In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src import parse
from collections import Counter
import random

import logging
import sys


log = logging.getLogger()
log.setLevel(logging.DEBUG)

if not log.hasHandlers():
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    log.addHandler(handler)

/Users/andrew/src/otp-scheduler


In [4]:
# ASSUMPTIONS
MEAL_CAPACITY = 26500
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 70 # not used, though

# This can be set a value from 0 (no effect) to the total count of restaurants (large effect)
RESTAURANT_FOUNDER_KNOB = 2
# Sometimes you may need to fine tune the meal steal knob above if one step is too big of a leap.
# This throws in randomness (0-1) for how often we trigger the meal steal logic (whether or not there is a meal steal opportunity)
RESTAURANT_FOUNDER_RANDOM_KNOB = 0.45

curr_capacity = MEAL_CAPACITY

def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

restaurants, num_zones = parse.parse_from_restaurants_csv('../../examples/boston/20200504/restaurants.csv')
restaurants.set_index('code', inplace=True)
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200504/hospitals.csv')).set_index('name')
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200504/vips.csv')).set_index('name')
vip_schedule = vip_hospitals.iloc[:, 4:]

assert hospitals['zone'].max() <= num_zones, 'A hospital is in a zone that we didn\'t expect from the restaurant sheet!'

rest_usage = restaurants.copy()
rest_usage.iloc[:, 3:(rest_usage.shape[1] - num_zones)] = 0

founding_restaurants = restaurants[restaurants['priority'] == 1].index.values

rest_cap = restaurants.copy()

total_hospitals = hospitals.shape[0]

print('Number of Restaurants:', restaurants.shape[0])
print('Founding (Priority == 1) Restaurants:', founding_restaurants.tolist())
print('Number of Hospitals:', total_hospitals)
print('Number of Zones:', num_zones)

Number of Restaurants: 21
Founding (Priority == 1) Restaurants: ['PG']
Number of Hospitals: 46
Number of Zones: 5


In [233]:
def day_from_meal(meal):
    return meal.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')

meal_log = None
meal_log_key_kludge = set()

def get_meal_log(schedule, is_vip):
    log = []
    for name, row in schedule.iterrows():
        for j, val in enumerate(row[4:]):
            # Add only if this cell is not blank, AND we haven't already added this hospital/meal.
            # This is possible if this order exists in both the regular and VIP meal schedule.
            # Since we add the VIP meals first, then we're guaranteed for it to be counted as a VIP meal if it's in both places.
            if not pd.isnull(val) and (name, schedule.columns[j + 4]) not in meal_log_key_kludge:
                log.append([
                    name,
                    row['zone'],
                    row['priority_rank'],
                    is_vip,
                    schedule.columns[j + 4],
                    val, 
                    np.nan,
                    day_from_meal(schedule.columns[j + 4]),
                    False
                ])
                meal_log_key_kludge.add((name, schedule.columns[j + 4]))
    return pd.DataFrame(log, columns=['hospital', 'zone', 'priority_rank', 'vip', 'meal', 'quantity', 'restaurant', 'day', 'failed'])

vip_log = get_meal_log(vip_hospitals, True)
meal_log = get_meal_log(hospitals, False).append(vip_log)

In [234]:
log.info(meal_log.shape)

INFO - (364, 9)


In [235]:
def get_rest_cap_cluster_for_meal(meal, zone):
    z = 'zone_' + str(zone)
    r = rest_cap[rest_cap[z] > 0][['priority', meal, z]]
    return r

In [236]:
def get_oustanding_vip_meals():
    return meal_log[(meal_log['vip'] == True) & meal_log['restaurant'].isnull()]

def get_outstanding_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone) & meal_log['restaurant'].isnull()]

def get_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone)]

def is_restaurant_serving_same_day_other_meal(rest_code, meal):
    day = day_from_meal(meal)
    rest_serving = meal_log[(meal_log['restaurant'] == rest_code) & (meal_log['day'] == day) & meal_log['meal'] != meal]
    return not rest_serving.empty

def assign_hospital_order(row):
    global curr_capacity

    meal = row['meal']
    hospital = row['hospital']
    zone = row['zone']
    
    print('Assigning', hospital, meal)
    
    # Check to see if we've already done this (sanity)
    if not pd.isnull(row['restaurant']):
        print('We have already assigned this one!')
        return False
        
    hospitals_in_cluster = get_hospitals_in_cluster_for_meal(meal, zone)

    # Restaurant capacity
    r_cap = get_rest_cap_cluster_for_meal(meal, zone)
        
    # The amount have to serve
    target_quantity = row['quantity']
    
    if curr_capacity - target_quantity < 0:
        print('We cannot to afford this meal because of our global limit!', curr_capacity, target_quantity)
        return False
    
    print(hospital, meal, zone, target_quantity)
    
    def find_restaurant():
        # Check if a restaurant is already serving the cluster
        restaurants_serving_cluster = hospitals_in_cluster[hospitals_in_cluster['restaurant'].notnull()]['restaurant']
        
        if not restaurants_serving_cluster.empty:
            # Filter out any that are already at the dropout limit
            rest_counter = Counter(restaurants_serving_cluster)
            rest_cluster_keys = rest_counter.keys()
            print('Checking for restaurants already in cluster', rest_counter)
            rest_with_room = [c[0] for c in rest_counter.most_common() if c[1] < MAX_DROPOFFS_PER_RUN]
            
            # Find restaurants with capacity
            candidates_with_quantity = []
            for r in rest_with_room:
                q = rest_cap.loc[r, meal]
                if q >= target_quantity:
                    print('Found an existing restaurant with capacity', r, q)
                    candidates_with_quantity.append(r)
                    if is_restaurant_serving_same_day_other_meal(r, meal):
                        print('Returning a restaurant with capacity, working on the same day', r, q)
                        return r
            if len(candidates_with_quantity):
                print('Returning a restaurant with capacity', r, q)
                return candidates_with_quantity[0]
        
        # Couldn't find an appropriate restaurant already serving cluster
        print('No suitable restaurants already working this cluster')
        
        # Find restaurants that are already serving the same meal time but in a different zone
        restaurants_in_other_zones = meal_log[
            (meal_log['meal'] == meal) & (
                meal_log['zone'] != zone) & (
                meal_log['restaurant'].notnull())]['restaurant'].unique()
                
        print('filtering down candidates with three lists', restaurants_serving_cluster, restaurants_in_other_zones)
        r_candidates = r_cap[
            (~r_cap.index.isin(restaurants_serving_cluster)) & (
                ~r_cap.index.isin(restaurants_in_other_zones)) & (
                r_cap[meal] >= target_quantity)
        ]
        
        print('Number of candidates with capacity', r_candidates.shape[0])
        
        if r_candidates.empty:
            return None
        
        # TODO add is_restaurant_serving_same_day_other_meal logic down here too!
        
        zone_col = 'zone_' + str(zone)

        # Sort in zone preference + (maybe) priority + quantity order (TODO KNOB: QUANTITY VAL USED TO BE ascending=False)
        if random.random() <= RESTAURANT_FOUNDER_RANDOM_KNOB:
            r_candidates = r_candidates.sort_values(by=[zone_col, meal], ascending=[False, True])
        else:
            r_candidates = r_candidates.sort_values(by=[zone_col, priority, meal], ascending=[False, True, True])
        
        day = day_from_meal(meal)
        
        # TODO Make sure restaurant isn't overbooked on their max day count
        final_valid_list = []
        for i, r in r_candidates.iterrows():
            print(i)
            days_worked = set(meal_log[meal_log['restaurant'] == i]['day'])
            # Check if we're already working this day, or the new day is still under cap
            if day in days_worked or len(days_worked) < rest_cap.loc[i, 'max_days_per_week']:
                # return i
                final_valid_list.append(i)
        
        # MEAL STEAL -- We need to prioritize founding Restaurants (like "PG, LD" in the Boston case)
        # TODO -- parameterize this, both the restaurants we want to do this for, and also how much we crank this knob
        # Comment out this section to ignore
        if len(final_valid_list) == 0:
            return None
        else:
            if len(final_valid_list) == 1:
                return final_valid_list[0]
            else:
                if RESTAURANT_FOUNDER_RANDOM_KNOB < 1 and random.random() <= RESTAURANT_FOUNDER_RANDOM_KNOB:
                    print('Entering potential meal steal', final_valid_list[0:RESTAURANT_FOUNDER_KNOB])
                    for r in final_valid_list[0:RESTAURANT_FOUNDER_KNOB]:
                        if r in founding_restaurants:
                            print('Meal Steal happened!', r)
                            return r
                return final_valid_list[0]
        
        return None
          
    r_code = find_restaurant()
    
    if r_code is None:
        print('CANNOT SERVE HOSPITAL FOR THEIR REQUEST', hospital, meal, r_cap)
        return False
                
    # Allocate the restaurant to the hospital(s)
    print('debiting r capacity: ', r_code, rest_cap.loc[r_code, meal])
    rest_cap.loc[r_code, meal] -= target_quantity
            
    print('increasing r usage', r_code, target_quantity)
    rest_usage.loc[r_code, meal] += target_quantity
        
    curr_capacity -= target_quantity
                
    print('updating log', hospital, meal, r_code)
    meal_log.loc[(meal_log['hospital'] == hospital) & (meal_log['meal'] == meal), 'restaurant'] = r_code
    
    return True

# HANDLE VIPS
for i, row in get_oustanding_vip_meals().iterrows():
    assigned = assign_hospital_order(row)
    if not assigned:
        meal_log.loc[(meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 'failed'] = True
    print()
    print() 

Assigning Brigham and Women's Hospital - ED lunch_wed
Brigham and Women's Hospital - ED lunch_wed 1 100.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: float64) []
Number of candidates with capacity 17
FV
ME
RT
CE
MY
SH
PO
BB
BC
CL
FC
RD
LD
MD
JC
SU
PG
debiting r capacity:  FV 100.0
increasing r usage FV 100.0
updating log Brigham and Women's Hospital - ED lunch_wed FV


Assigning Brigham and Women's Hospital - ED lunch_sun
Brigham and Women's Hospital - ED lunch_sun 1 100.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 10
MY
SH
BB
CL
NT
LD
MD
JC
SU
PG
debiting r capacity:  MY 200.0
increasing r usage MY 100.0
updating log Brigham and Women's Hospital - ED lunch_sun MY


Assigning Brigham and Women's Hospital - ED dinner_wed
Brigham and Women's Hospital - ED dinner_wed 1 

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)



Assigning Brigham & Women's Hospital (COVID ICUs) lunch_thurs
Brigham & Women's Hospital (COVID ICUs) lunch_thurs 1 210.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 13
SH
PO
BB
BC
CL
FC
RD
NT
LD
MD
JC
SU
PG
debiting r capacity:  SH 225.0
increasing r usage SH 210.0
updating log Brigham & Women's Hospital (COVID ICUs) lunch_thurs SH


Assigning Brigham & Women's Hospital (COVID ICUs) lunch_fri
Brigham & Women's Hospital (COVID ICUs) lunch_fri 1 210.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 11
SH
PO
BB
BC
CL
FC
NT
MD
JC
SU
PG
debiting r capacity:  SH 225.0
increasing r usage SH 210.0
updating log Brigham & Women's Hospital (COVID ICUs) lunch_fri SH


Assigning Brigham & Women's Hospital (COVID ICUs) lunch_sat
Brigham &

Beth Israel Deaconess Medical Center - West Campus lunch_tues 1 180.0
Checking for restaurants already in cluster Counter({'SH': 1, 'TB': 1})
No suitable restaurants already working this cluster
filtering down candidates with three lists 5     SH
23    TB
Name: restaurant, dtype: object []
Number of candidates with capacity 11
MY
PO
BB
BC
CL
FC
LD
MD
JC
SU
PG
Entering potential meal steal ['MY', 'PO']
debiting r capacity:  MY 200.0
increasing r usage MY 180.0
updating log Beth Israel Deaconess Medical Center - West Campus lunch_tues MY


Assigning Beth Israel Deaconess Medical Center - West Campus lunch_wed
Beth Israel Deaconess Medical Center - West Campus lunch_wed 1 180.0
Checking for restaurants already in cluster Counter({'FV': 1, 'SH': 1, 'TB': 1})
No suitable restaurants already working this cluster
filtering down candidates with three lists 0     FV
6     SH
24    TB
Name: restaurant, dtype: object []
Number of candidates with capacity 12
MY
PO
BB
BC
CL
FC
RD
LD
MD
JC
SU
PG
deb

BWH - Faulkner 7N Covid Unit lunch_wed 2 20.0
Checking for restaurants already in cluster Counter({'ME': 1, 'RT': 1})
Found an existing restaurant with capacity ME 105.0
Returning a restaurant with capacity, working on the same day ME 105.0
debiting r capacity:  ME 105.0
increasing r usage ME 20.0
updating log BWH - Faulkner 7N Covid Unit lunch_wed ME


Assigning BWH - Faulkner 7N Covid Unit lunch_thurs
BWH - Faulkner 7N Covid Unit lunch_thurs 2 20.0
Checking for restaurants already in cluster Counter({'ME': 1})
No suitable restaurants already working this cluster
filtering down candidates with three lists 46    ME
Name: restaurant, dtype: object ['SH' 'TB' 'MY']
Number of candidates with capacity 14
FV
CE
PO
BB
BC
CL
FC
RD
NT
LD
MD
JC
SU
PG
Entering potential meal steal ['FV', 'CE']
debiting r capacity:  FV 100.0
increasing r usage FV 20.0
updating log BWH - Faulkner 7N Covid Unit lunch_thurs FV


Assigning BWH - Faulkner 7N Covid Unit lunch_fri
BWH - Faulkner 7N Covid Unit lunch_fri 

BC
CL
FC
LD
MD
JC
SU
PG
Entering potential meal steal ['PO', 'BB']
debiting r capacity:  PO 300.0
increasing r usage PO 12.0
updating log Boston Healthcare for the Homeless - Tents lunch_tues PO


Assigning Boston Healthcare for the Homeless - Tents lunch_wed
Boston Healthcare for the Homeless - Tents lunch_wed 3 12.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['FV' 'SH' 'TB' 'MY' 'ME' 'RT']
Number of candidates with capacity 12
CE
PO
BB
BC
CL
FC
RD
LD
MD
JC
SU
PG
debiting r capacity:  CE 150.0
increasing r usage CE 12.0
updating log Boston Healthcare for the Homeless - Tents lunch_wed CE


Assigning Boston Healthcare for the Homeless - Tents lunch_thurs
Boston Healthcare for the Homeless - Tents lunch_thurs 3 12.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['SH' 'TB' 'MY' 'ME' 'FV']
Number of c

Checking for restaurants already in cluster Counter({'BB': 1})
No suitable restaurants already working this cluster
filtering down candidates with three lists 94    BB
Name: restaurant, dtype: object ['SH' 'FC' 'ME']
Number of candidates with capacity 4
LD
MD
SU
PG
Entering potential meal steal ['LD', 'MD']
debiting r capacity:  LD 400.0
increasing r usage LD 400.0
updating log Boston Medical Center dinner_thurs LD


Assigning Boston Medical Center dinner_sat
Boston Medical Center dinner_sat 3 150.0
Checking for restaurants already in cluster Counter({'MM': 1})
No suitable restaurants already working this cluster
filtering down candidates with three lists 96    MM
Name: restaurant, dtype: object ['SH' 'BB' 'FC']
Number of candidates with capacity 5
RD
NT
MD
SU
PG
debiting r capacity:  RD 300.0
increasing r usage RD 150.0
updating log Boston Medical Center dinner_sat RD


Assigning Boston Medical Center dinner_sun
Boston Medical Center dinner_sun 3 150.0
Checking for restaurants already

PG
Entering potential meal steal ['BB', 'BC']
debiting r capacity:  BB 300.0
increasing r usage BB 30.0
updating log Massachusetts General Hospital - PICU lunch_thurs BB


Assigning Massachusetts General Hospital - PICU lunch_fri
Massachusetts General Hospital - PICU lunch_fri 4 30.0
Checking for restaurants already in cluster Counter({'BB': 1})
Found an existing restaurant with capacity BB 150.0
Returning a restaurant with capacity, working on the same day BB 150.0
debiting r capacity:  BB 150.0
increasing r usage BB 30.0
updating log Massachusetts General Hospital - PICU lunch_fri BB


Assigning Massachusetts General Hospital - PICU dinner_tues
Massachusetts General Hospital - PICU dinner_tues 4 20.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['SH' 'CS' 'FC' 'ME' 'LD']
Number of candidates with capacity 6
BB
BC
RD
MD
SU
PG
debiting r capacity:  BB 300.0
increasing r usage BB 20.0
updating

FC
RD
MD
JC
SU
PG
debiting r capacity:  BB 300.0
increasing r usage BB 120.0
updating log Cambridge Health Alliance - Cambridge Hospital lunch_wed BB


Assigning Cambridge Health Alliance - Cambridge Hospital lunch_fri
Cambridge Health Alliance - Cambridge Hospital lunch_fri 5 120.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['SH' 'ME' 'MY' 'RT' 'CE' 'PO' 'MD' 'BB']
Number of candidates with capacity 7
BC
CL
FC
NT
JC
SU
PG
debiting r capacity:  BC 300.0
increasing r usage BC 120.0
updating log Cambridge Health Alliance - Cambridge Hospital lunch_fri BC


Assigning Cambridge Health Alliance - Cambridge Hospital lunch_sun
Cambridge Health Alliance - Cambridge Hospital lunch_sun 5 120.0
No suitable restaurants already working this cluster
filtering down candidates with three lists Series([], Name: restaurant, dtype: object) ['MY' 'SH' 'PG' 'TB' 'BB' 'CL' 'NT']
Number of candidates with capacit

In [237]:
meal_log[(meal_log.vip == True) & (meal_log.failed == True)]

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed


In [238]:
hospitals_that_failed_placement = set()

def select_and_assign_hospital():
    next_hospital = meal_log[~meal_log.hospital.isin(list(hospitals_that_failed_placement))].groupby('hospital')[['priority_rank', 'restaurant']].agg({
            'restaurant': 'count',
            'priority_rank': 'min'
        }).sort_values(by=['restaurant', 'priority_rank']).reset_index()
    
    if next_hospital.shape[0] > 0:
        next_hospital = next_hospital.iloc[0]
    else:
        return False
    
    print('Next hospital:', next_hospital)

    # Find potential orders to serve, ordered by quantity
    possible_orders = meal_log[(meal_log['hospital'] == next_hospital['hospital']) & (meal_log['restaurant'].isnull()) & (meal_log['failed'] == False)].sort_values(by='quantity', ascending=False)
    
    print('Possible orders', possible_orders)
    for i, row in possible_orders.iterrows():
        if assign_hospital_order(row):
            return True
        else:
            meal_log.loc[
                (meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 
                'failed'
            ] = True
            
    hospitals_that_failed_placement.add(next_hospital['hospital'])
    return True
    

while curr_capacity > 0 and len(hospitals_that_failed_placement) < total_hospitals:
    retval = select_and_assign_hospital()
    if retval == False:
        break
    print()
    print()

Next hospital: hospital         Boston Children's Hospital - ED
restaurant                                     0
priority_rank                                  1
Name: 0, dtype: object
Possible orders                           hospital  zone  priority_rank    vip          meal  \
0  Boston Children's Hospital - ED     1              1  False    dinner_mon   
1  Boston Children's Hospital - ED     1              1  False   dinner_tues   
2  Boston Children's Hospital - ED     1              1  False    dinner_wed   
3  Boston Children's Hospital - ED     1              1  False  dinner_thurs   
4  Boston Children's Hospital - ED     1              1  False    dinner_fri   

   quantity restaurant    day  failed  
0      60.0        NaN    mon   False  
1      60.0        NaN   tues   False  
2      60.0        NaN    wed   False  
3      60.0        NaN  thurs   False  
4      60.0        NaN    fri   False  
Assigning Boston Children's Hospital - ED dinner_mon
Boston Children's Hospita


Next hospital: hospital         Bowdoin Street Health Center
restaurant                                  0
priority_rank                               2
Name: 0, dtype: object
Possible orders                         hospital  zone  priority_rank    vip       meal  \
77  Bowdoin Street Health Center     2              2  False  lunch_mon   
78  Bowdoin Street Health Center     2              2  False  lunch_wed   
79  Bowdoin Street Health Center     2              2  False  lunch_fri   

    quantity restaurant  day  failed  
77      52.0        NaN  mon   False  
78      52.0        NaN  wed   False  
79      52.0        NaN  fri   False  
Assigning Bowdoin Street Health Center lunch_mon
Bowdoin Street Health Center lunch_mon 2 52.0
Checking for restaurants already in cluster Counter({'CE': 2, 'RT': 1})
Found an existing restaurant with capacity CE 100.0
Returning a restaurant with capacity, working on the same day CE 100.0
debiting r capacity:  CE 100.0
increasing r usage CE 52.0
up

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value

Next hospital: hospital         Charles River Community Health
restaurant                                    0
priority_rank                                 2
Name: 0, dtype: object
Possible orders                            hospital  zone  priority_rank    vip         meal  \
197  Charles River Community Health     5              2  False    lunch_mon   
198  Charles River Community Health     5              2  False   lunch_tues   
199  Charles River Community Health     5              2  False    lunch_wed   
200  Charles River Community Health     5              2  False  lunch_thurs   
201  Charles River Community Health     5              2  False    lunch_fri   

     quantity restaurant    day  failed  
197      34.0        NaN    mon   False  
198      34.0        NaN   tues   False  
199      34.0        NaN    wed   False  
200      34.0        NaN  thurs   False  
201      34.0        NaN    fri   False  
Assigning Charles River Community Health lunch_mon
Charles River Comm

CL
FC
MD
JC
SU
debiting r capacity:  BB 300.0
increasing r usage BB 100.0
updating log East Boston Neighborhood Health Center lunch_tues BB


Next hospital: hospital         Harbor Health Services Inc.
restaurant                                 0
priority_rank                              2
Name: 0, dtype: object
Possible orders                         hospital  zone  priority_rank    vip       meal  \
101  Harbor Health Services Inc.     2              2  False  lunch_wed   
102  Harbor Health Services Inc.     2              2  False  lunch_fri   

     quantity restaurant  day  failed  
101      25.0        NaN  wed   False  
102      25.0        NaN  fri   False  
Assigning Harbor Health Services Inc. lunch_wed
Harbor Health Services Inc.

...

 hospital         Tufts Medical Center - Cardiothoracic ICU
restaurant                                              10
priority_rank                                            1
Name: 0, dtype: object
Possible orders Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         Pine Street Inn - Site 1
restaurant                             10
priority_rank                           2
Name: 0, dtype: object
Possible orders                      hospital  zone  priority_rank    vip         meal  \
138  Pine Street Inn - Site 1     3              2  False    lunch_wed   
139  Pine Street Inn - Site 1     3              2  False  lunch_thurs   
140  Pine Street Inn - Site 1     3              2  False    lunch_fri   

     quantity restaurant    day  failed  
138      15.0        NaN    wed   False  
139      15.0        NaN  thurs   False  
140      15.0        NaN    fri   False  
Assigning Pine Street I

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


 74.0 135.0


Next hospital: hospital         Pine Street Inn - Site 1
restaurant                             11
priority_rank                           2
Name: 0, dtype: object
Possible orders                      hospital  zone  priority_rank    vip         meal  \
139  Pine Street Inn - Site 1     3              2  False  lunch_thurs   
140  Pine Street Inn - Site 1     3              2  False    lunch_fri   

     quantity restaurant    day  failed  
139      15.0        NaN  thurs   False  
140      15.0        NaN    fri   False  
Assigning Pine Street Inn - Site 1 lunch_thurs
Pine Street Inn - Site 1 lunch_thurs 3 15.0
Checking for restaurants already in cluster Counter({'PO': 4, 'JC': 3, 'LD': 1})
Found an existing restaurant with capacity JC 250.0
Returning a restaurant with capacity, working on the same day JC 250.0
debiting r capacity:  JC 250.0
increasing r usage JC 15.0
updating log Pine Street Inn - Site 1 lunch_thurs JC


Next hospital: hospital         Pine Street Inn -

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


 lunch_fri 3 15.0
Checking for restaurants already in cluster Counter({'PO': 4, 'JC': 3, 'MD': 1})
Found an existing restaurant with capacity JC 250.0
Returning a restaurant with capacity, working on the same day JC 250.0
debiting r capacity:  JC 250.0
increasing r usage JC 15.0
updating log Pine Street Inn - Site 1 lunch_fri JC


Next hospital: hospital         Pine Street Inn - Site 2
restaurant                             12
priority_rank                           2
Name: 0, dtype: object
Possible orders                      hospital  zone  priority_rank    vip       meal  \
155  Pine Street Inn - Site 2     3              2  False  lunch_sat   
156  Pine Street Inn - Site 2     3              2  False  lunch_sun   

     quantity restaurant  day  failed  
155      10.0        NaN  sat   False  
156      10.0        NaN  sun   False  
Assigning Pine Street Inn - Site 2 lunch_sat
Pine Street Inn - Site 2 lunch_sat 3 10.0
Checking for restaurants already in cluster Counter({'CL': 4})


In [239]:
# Remaining capacity
curr_capacity

0.0

In [240]:
# Fullfilled (this should be == global capacity - curr_capacity)
meal_log[meal_log['restaurant'].notnull()]['quantity'].sum()

26500.0

In [253]:
# Missed
meal_log[meal_log['restaurant'].isnull()]['quantity'].sum()

4604.0

In [241]:
# Create empty results frame
output = hospitals.copy()
output.iloc[:, 4:] = np.nan


# construct the double-wide output dataframe
for i, c in enumerate(hospitals.columns[4:]):
    output.insert((i*2+1)+4, c + '_rest', np.nan)
        
for i, row in meal_log.iterrows():
    output.loc[row['hospital'], row['meal']] = row['quantity']
    if not pd.isnull(row['restaurant']):
        output.loc[row['hospital'], row['meal'] + '_rest'] = row['restaurant']
    else:
        output.loc[row['hospital'], row['meal'] + '_rest'] = 'XXX'


# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         val = str(row['restaurant'] + ' - ' + str(row['quantity']))
#     else:
#         val = str('XXX - ' + str(row['quantity']))
#     output.loc[row['hospital'], row['meal']] = val
    
# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         output.loc[row['hospital'], row['meal']] = row['restaurant']
        
output.to_csv('../../output.csv')


In [242]:
output

,code,priority_rank,region,zone,breakfast_mon,breakfast_mon_rest,breakfast_tues,breakfast_tues_rest,breakfast_wed,breakfast_wed_rest,...,dinner_wed,dinner_wed_rest,dinner_thurs,dinner_thurs_rest,dinner_fri,dinner_fri_rest,dinner_sat,dinner_sat_rest,dinner_sun,dinner_sun_rest
name,,,,,,,,,,,,,,,,,,,,,
Brigham and Women's Hospital - ED,4,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,CS,NaN,NaN,NaN,NaN,NaN,NaN,100.0,MM
Boston Children's Hospital - ED,18,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,60.0,ME,60.0,BC,60.0,NT,NaN,NaN,NaN,NaN
Dana Farber Cancer Institute - Inpatient PAs,23,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brigham & Women's Hospital (COVID ICUs),36,1,Longwood,1,210.0,XXX,210.0,XXX,210.0,XXX,...,210.0,SH,210.0,SH,210.0,SH,200.0,SH,200.0,SH
Brigham and Women's Hospital - Central,52,1,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,300.0,BB,300.0,BB
Beth Israel Deaconess Medical Center - East Campus,53,1,Longwood,1,55.0,XXX,55.0,XXX,55.0,XXX,...,140.0,ME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beth Israel Deaconess Medical Center - West Campus,54,1,Longwood,1,NaN,NaN,200.0,XXX,NaN,NaN,...,NaN,NaN,100.0,FC,NaN,NaN,NaN,NaN,NaN,NaN
Whittier Street Health Center,15,2,Longwood,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BWH - Faulkner ED,5,1,Roxbury / Jamaica Plain,2,NaN,NaN,NaN,NaN,NaN,NaN,...,45.0,FC,NaN,NaN,NaN,NaN,NaN,NaN,45.0,NT


In [243]:
rest_usage.iloc[:, 3:(rest_usage.shape[1] - num_zones)].sum(axis=1).sort_values(ascending=False).sum()

26500.0

In [245]:
equity = meal_log.groupby(['priority_rank', 'hospital'])[['meal', 'vip', 'restaurant']].agg({
    'restaurant': [('orders_fulfilled', 'count')],
    'meal': [('orders_requested', 'count')],
    'vip': [('vip_orders', 'sum')],
})
equity

restaurant  \
                                                                 orders_fulfilled   
priority_rank hospital                                                              
1             BWH - Faulkner 7N Covid Unit                                     14   
              BWH - Faulkner ED                                                 4   
              BWH - Faulkner Hospital Central                                   2   
              BWH - Faulkner ICU                                               14   
              Beth Israel Deaconess Medical Center - East Campus                8   
              Beth Israel Deaconess Medical Center - West Campus                9   
              Boston Children's Hospital - ED                                   5   
              Boston EMS - South End                                            5   
              Boston Healthcare for the Homeless - Albany Street               14   
              Boston Healthcare for the Homeless - Tents                       14   
              Boston Medical Center                                            11   
              Brigham & Women's Hospital (COVID ICUs)                          14   
              Brigham and Women's Hospital - Central                            4   
              Brigham and Women's Hospital - ED                                 4   
              Cambridge Health Alliance - Cambridge Hospital                   11   
              Cambridge Health Alliance - Everett Hospital                     11   
              Carney Hospital - ED                                             11   
              Dana Farber Cancer Institute - Inpatient PAs                      7   
              Massachusetts General Hospital - Central                          7   
              Massachusetts General Hospital - PICU                             7   
              Mt. Auburn Hospital                                               1   
              Newton Wellesley Hospital                                         1   
              Tufts Medical Center                                             10   
              Tufts Medical Center - Cardiothoracic ICU                        10   
              VA Medical Center - West Roxbury                                  6   
2             BIDMC - Chelsea                                                   5   
              Bowdoin Street Health Center                                      3   
              Charles River Community Health                                    5   
              Codman Square Health Center - Site 1                              5   
              Codman Square Health Center - Site 2                              5   
              Dimock Center                                                     4   
              Dot House Health Dorchester                                       5   
              East Boston Neighborhood Health Center                            6   
              Harbor Health Services Inc.                                       2   
              Harvard Street Neighborhood Health Center                         5   
              Mattapan Community Health Center                                  3   
              North End Waterfront Health Center                                4   
              Pine Street Inn - Site 1                                         13   
              Pine Street Inn - Site 2                                         14   
              Pine Street Inn Holy Family Shelter                               7   
              Pine Street Inn Shattuck Shelter                                 13   
              South Boston Community Health Center                              5   
              South End Community Health Center                                 5   
              Southern Jamaica Plain Health Center (part of B...                5   
              Upham's Corner Health Center                                  

In [246]:
equity.to_csv('../../equity.csv')

In [247]:
meal_log[meal_log['hospital'] == 'Boston Medical Center']

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
103,Boston Medical Center,3,1,False,dinner_mon,400.0,NaN,mon,True
104,Boston Medical Center,3,1,False,dinner_wed,400.0,NaN,wed,True
105,Boston Medical Center,3,1,False,dinner_fri,400.0,NaN,fri,True
98,Boston Medical Center,3,1,True,lunch_mon,400.0,LD,mon,False
99,Boston Medical Center,3,1,True,lunch_tues,400.0,LD,tues,False
100,Boston Medical Center,3,1,True,lunch_wed,400.0,LD,wed,False
101,Boston Medical Center,3,1,True,lunch_thurs,400.0,LD,thurs,False
102,Boston Medical Center,3,1,True,lunch_fri,400.0,MD,fri,False
103,Boston Medical Center,3,1,True,lunch_sat,150.0,CL,sat,False
104,Boston Medical Center,3,1,True,lunch_sun,150.0,CL,sun,False


In [252]:
# rest_usage.iloc[:, 3:-num_zones].sum(axis=1).sort_values(ascending=False)

In [250]:
rest_stats = meal_log[['restaurant', 'quantity']].groupby('restaurant').agg({
    'quantity': ['sum', 'count', 'mean', 'min', 'max']
})
rest_stats.columns = ['sum_total', 'count', 'mean', 'min', 'max']
rest_stats = rest_stats.sort_values(by='sum_total', ascending=False)
rest_stats

,sum_total,count,mean,min,max
restaurant,,,,,
BB,2996.0,41,73.073171,4.0,300.0
LD,2974.0,15,198.266667,12.0,400.0
SH,2895.0,17,170.294118,10.0,210.0
PG,2877.0,15,191.800000,4.0,550.0
MD,1994.0,32,62.312500,10.0,400.0
PO,1278.0,20,63.900000,12.0,150.0
MY,1276.0,10,127.600000,6.0,200.0
BC,1268.0,20,63.400000,12.0,150.0
SU,1247.0,6,207.833333,7.0,500.0


In [251]:
rest_stats.to_csv('../../rest_stats.csv')